In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense,RepeatVector
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D,Conv2D
from tensorflow.keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
import tensorflow
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [4]:
error = pd.DataFrame(columns=['File_name','RMSE_ERROR_CNN','RMSE_ERROR_LSTM','RMSE_ERROR_CNN+LSTM','MSE','MAPE'])

In [5]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
dff=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/Forecast_Error_CNN.csv")

In [9]:
from sklearn.model_selection import train_test_split
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten

In [10]:
import os
import logging, sys
logging.disable(sys.maxsize)
from math import sqrt
paths_to_folders = ['C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Splited Data s-1 to s-5/STORE__1']
x=0
for folder in paths_to_folders:
   for csv_file in os.listdir(folder):
        if  any(dff.File_name ==""+csv_file)==True:
       #if len(df.index)>=100:
            select_indices=list(np.where(dff["File_name"] == ''+csv_file)[0])
            print(select_indices)
            df=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Splited Data s-1 to s-5/STORE__1/"+csv_file,parse_dates=['date'])
            df.drop(['id','store_nbr','item_nbr','onpromotion'], axis=1,inplace=True)
            k = int(len(df) * 0.8)
            train = df[:k]  
            test= df[k:]
            lag_size = (test['date'].max().date() - train['date'].max().date()).days
            window =29
            lag = lag_size
            series = series_to_supervised(train.drop('date', axis=1), window=window, lag=lag)
            labels_col = 'unit_sales(t+%d)' % lag
            labels = series[labels_col]
            series = series.drop(labels_col, axis=1)
            X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.2, random_state=0)
            X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1],1))
            X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1],1))
            subsequences = 2
            timesteps = X_train_series.shape[1]//subsequences
            X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, timesteps, 1))
            X_valid_series_sub = X_valid_series.reshape((X_valid_series.shape[0], subsequences, timesteps, 1))
            model_cnn_lstm = Sequential()
            model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
            model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
            model_cnn_lstm.add(TimeDistributed(Flatten()))
            model_cnn_lstm.add(LSTM(50, activation='relu'))
            #model_cnn_lstm.add(LSTM(10, stateful=True))
            model_cnn_lstm.add(Dense(1))
            model_cnn_lstm.compile(loss='mse', optimizer='adam')
            cnn_lstm_history = model_cnn_lstm.fit(X_train_series_sub, Y_train, validation_data=(X_valid_series_sub, Y_valid), epochs=60, verbose=2)
            cnn_lstm_valid_pred = model_cnn_lstm.predict(X_valid_series_sub)
            
            rmse =  np.sqrt(mean_squared_error(Y_valid, cnn_lstm_valid_pred))
            error.loc[x, ['File_name']]=csv_file
            error.loc[x, ['RMSE_ERROR_CNN+LSTM']]=rmse
            if len(select_indices)==1:
                b=dff.iloc[select_indices[0]]['RMSE_ERROR_LSTM']
                d=dff.iloc[select_indices[0]]['RMSE_ERROR_CNN']
                error.loc[x, ['RMSE_ERROR_LSTM']]=b
                error.loc[x, ['RMSE_ERROR_CNN']]=d
            error.loc[x, ['MSE']]=mean_squared_error(Y_valid, cnn_lstm_valid_pred)
            error.loc[x, ['MAPE']]=mean_absolute_percentage_error(Y_valid, cnn_lstm_valid_pred)
            x=x+1
            print(x)

[0]
Train on 30 samples, validate on 8 samples
Epoch 1/60
 - 1s - loss: 8.1731 - val_loss: 6.5176
Epoch 2/60
 - 0s - loss: 6.7385 - val_loss: 5.0657
Epoch 3/60
 - 0s - loss: 5.4574 - val_loss: 3.7254
Epoch 4/60
 - 0s - loss: 4.3177 - val_loss: 2.6150
Epoch 5/60
 - 0s - loss: 3.4085 - val_loss: 1.7124
Epoch 6/60
 - 0s - loss: 2.7517 - val_loss: 1.3185
Epoch 7/60
 - 0s - loss: 2.6045 - val_loss: 1.5196
Epoch 8/60
 - 0s - loss: 2.9940 - val_loss: 1.8261
Epoch 9/60
 - 0s - loss: 3.3832 - val_loss: 1.8596
Epoch 10/60
 - 0s - loss: 3.4117 - val_loss: 1.6739
Epoch 11/60
 - 0s - loss: 3.1628 - val_loss: 1.4479
Epoch 12/60
 - 0s - loss: 2.8429 - val_loss: 1.3103
Epoch 13/60
 - 0s - loss: 2.5968 - val_loss: 1.3024
Epoch 14/60
 - 0s - loss: 2.4727 - val_loss: 1.3965
Epoch 15/60
 - 0s - loss: 2.4601 - val_loss: 1.5338
Epoch 16/60
 - 0s - loss: 2.5123 - val_loss: 1.6602
Epoch 17/60
 - 0s - loss: 2.5764 - val_loss: 1.7409
Epoch 18/60
 - 0s - loss: 2.6194 - val_loss: 1.7651
Epoch 19/60
 - 0s - loss: 

Epoch 37/60
 - 0s - loss: 0.6678 - val_loss: 0.8223
Epoch 38/60
 - 0s - loss: 0.6641 - val_loss: 0.8292
Epoch 39/60
 - 0s - loss: 0.6574 - val_loss: 0.8332
Epoch 40/60
 - 0s - loss: 0.6477 - val_loss: 0.8353
Epoch 41/60
 - 0s - loss: 0.6357 - val_loss: 0.8368
Epoch 42/60
 - 0s - loss: 0.6230 - val_loss: 0.8392
Epoch 43/60
 - 0s - loss: 0.6112 - val_loss: 0.8433
Epoch 44/60
 - 0s - loss: 0.6008 - val_loss: 0.8495
Epoch 45/60
 - 0s - loss: 0.5923 - val_loss: 0.8574
Epoch 46/60
 - 0s - loss: 0.5854 - val_loss: 0.8666
Epoch 47/60
 - 0s - loss: 0.5793 - val_loss: 0.8761
Epoch 48/60
 - 0s - loss: 0.5733 - val_loss: 0.8853
Epoch 49/60
 - 0s - loss: 0.5667 - val_loss: 0.8941
Epoch 50/60
 - 0s - loss: 0.5592 - val_loss: 0.9023
Epoch 51/60
 - 0s - loss: 0.5507 - val_loss: 0.9101
Epoch 52/60
 - 0s - loss: 0.5414 - val_loss: 0.9182
Epoch 53/60
 - 0s - loss: 0.5319 - val_loss: 0.9268
Epoch 54/60
 - 0s - loss: 0.5227 - val_loss: 0.9365
Epoch 55/60
 - 0s - loss: 0.5141 - val_loss: 0.9472
Epoch 56/60


Epoch 13/60
 - 0s - loss: 0.8827 - val_loss: 0.8742
Epoch 14/60
 - 0s - loss: 0.8575 - val_loss: 1.0382
Epoch 15/60
 - 0s - loss: 0.8735 - val_loss: 1.1741
Epoch 16/60
 - 0s - loss: 0.9044 - val_loss: 1.2723
Epoch 17/60
 - 0s - loss: 0.9325 - val_loss: 1.3258
Epoch 18/60
 - 0s - loss: 0.9469 - val_loss: 1.3346
Epoch 19/60
 - 0s - loss: 0.9436 - val_loss: 1.3021
Epoch 20/60
 - 0s - loss: 0.9241 - val_loss: 1.2361
Epoch 21/60
 - 0s - loss: 0.8928 - val_loss: 1.1460
Epoch 22/60
 - 0s - loss: 0.8561 - val_loss: 1.0420
Epoch 23/60
 - 0s - loss: 0.8211 - val_loss: 0.9345
Epoch 24/60
 - 0s - loss: 0.7940 - val_loss: 0.8333
Epoch 25/60
 - 0s - loss: 0.7793 - val_loss: 0.7465
Epoch 26/60
 - 0s - loss: 0.7772 - val_loss: 0.6794
Epoch 27/60
 - 0s - loss: 0.7833 - val_loss: 0.6340
Epoch 28/60
 - 0s - loss: 0.7909 - val_loss: 0.6105
Epoch 29/60
 - 0s - loss: 0.7931 - val_loss: 0.6071
Epoch 30/60
 - 0s - loss: 0.7862 - val_loss: 0.6225
Epoch 31/60
 - 0s - loss: 0.7709 - val_loss: 0.6543
Epoch 32/60


Epoch 50/60
 - 0s - loss: 3.6152 - val_loss: 0.4535
Epoch 51/60
 - 0s - loss: 3.5782 - val_loss: 0.4427
Epoch 52/60
 - 0s - loss: 3.5432 - val_loss: 0.4387
Epoch 53/60
 - 0s - loss: 3.5082 - val_loss: 0.4416
Epoch 54/60
 - 0s - loss: 3.4718 - val_loss: 0.4519
Epoch 55/60
 - 0s - loss: 3.4331 - val_loss: 0.4698
Epoch 56/60
 - 0s - loss: 3.3929 - val_loss: 0.4949
Epoch 57/60
 - 0s - loss: 3.3514 - val_loss: 0.5256
Epoch 58/60
 - 0s - loss: 3.3097 - val_loss: 0.5597
Epoch 59/60
 - 0s - loss: 3.2680 - val_loss: 0.5940
Epoch 60/60
 - 0s - loss: 3.2265 - val_loss: 0.6250
8
[8]
Train on 12 samples, validate on 3 samples
Epoch 1/60
 - 1s - loss: 1.7079 - val_loss: 6.8376
Epoch 2/60
 - 0s - loss: 1.2008 - val_loss: 5.9585
Epoch 3/60
 - 0s - loss: 0.7879 - val_loss: 5.2131
Epoch 4/60
 - 0s - loss: 0.5029 - val_loss: 4.4629
Epoch 5/60
 - 0s - loss: 0.3137 - val_loss: 3.7519
Epoch 6/60
 - 0s - loss: 0.2583 - val_loss: 3.2826
Epoch 7/60
 - 0s - loss: 0.3400 - val_loss: 3.1100
Epoch 8/60
 - 0s - los

Epoch 26/60
 - 0s - loss: 1.6915 - val_loss: 1.5200
Epoch 27/60
 - 0s - loss: 1.6589 - val_loss: 1.6876
Epoch 28/60
 - 0s - loss: 1.6440 - val_loss: 1.8620
Epoch 29/60
 - 0s - loss: 1.6434 - val_loss: 2.0157
Epoch 30/60
 - 0s - loss: 1.6487 - val_loss: 2.1274
Epoch 31/60
 - 0s - loss: 1.6501 - val_loss: 2.1819
Epoch 32/60
 - 0s - loss: 1.6404 - val_loss: 2.1767
Epoch 33/60
 - 0s - loss: 1.6181 - val_loss: 2.1218
Epoch 34/60
 - 0s - loss: 1.5874 - val_loss: 2.0346
Epoch 35/60
 - 0s - loss: 1.5548 - val_loss: 1.9348
Epoch 36/60
 - 0s - loss: 1.5267 - val_loss: 1.8376
Epoch 37/60
 - 0s - loss: 1.5061 - val_loss: 1.7566
Epoch 38/60
 - 0s - loss: 1.4934 - val_loss: 1.6963
Epoch 39/60
 - 0s - loss: 1.4860 - val_loss: 1.6591
Epoch 40/60
 - 0s - loss: 1.4805 - val_loss: 1.6452
Epoch 41/60
 - 0s - loss: 1.4738 - val_loss: 1.6540
Epoch 42/60
 - 0s - loss: 1.4638 - val_loss: 1.6846
Epoch 43/60
 - 0s - loss: 1.4499 - val_loss: 1.7359
Epoch 44/60
 - 0s - loss: 1.4332 - val_loss: 1.8061
Epoch 45/60


...

 - 0s - loss: 2.2697 - val_loss: 2.2579
Epoch 20/60
 - 0s - loss: 2.2472 - val_loss: 2.1835
Epoch 21/60
 - 0s - loss: 2.2034 - val_loss: 2.0689
Epoch 22/60
 - 0s - loss: 2.1461 - val_loss: 1.9284
Epoch 23/60
 - 0s - loss: 2.0845 - val_loss: 1.7777
Epoch 24/60
 - 0s - loss: 2.0283 - val_loss: 1.6321
Epoch 25/60
 - 0s - loss: 1.9854 - val_loss: 1.5043
Epoch 26/60
 - 0s - loss: 1.9595 - val_loss: 1.4027
Epoch 27/60
 - 0s - loss: 1.9495 - val_loss: 1.3302
Epoch 28/60
 - 0s - loss: 1.9489 - val_loss: 1.2850
Epoch 29/60
 - 0s - loss: 1.9480 - val_loss: 1.2628
Epoch 30/60
 - 0s - loss: 1.9385 - val_loss: 1.2598
Epoch 31/60
 - 0s - loss: 1.9161 - val_loss: 1.2735
Epoch 32/60
 - 0s - loss: 1.8822 - val_loss: 1.3032
Epoch 33/60
 - 0s - loss: 1.8417 - val_loss: 1.3474
Epoch 34/60
 - 0s - loss: 1.8038 - val_loss: 1.4021
Epoch 35/60
 - 0s - loss: 1.7727 - val_loss: 1.4607
Epoch 36/60
 - 0s - loss: 1.7480 - val_loss: 1.5170
Epoch 37/60
 - 0s - loss: 1.7287 - val_loss: 1.5638
Epoch 38/60
 - 0s - loss

Epoch 56/60
 - 0s - loss: 0.4165 - val_loss: 1.8895
Epoch 57/60
 - 0s - loss: 0.4106 - val_loss: 1.9663
Epoch 58/60
 - 0s - loss: 0.4055 - val_loss: 2.0470
Epoch 59/60
 - 0s - loss: 0.4026 - val_loss: 2.1144
Epoch 60/60
 - 0s - loss: 0.4011 - val_loss: 2.1582
91
[91]
Train on 40 samples, validate on 11 samples
Epoch 1/60
 - 9s - loss: 8.5457 - val_loss: 7.7739
Epoch 2/60
 - 0s - loss: 6.6096 - val_loss: 5.5178
Epoch 3/60
 - 0s - loss: 4.8974 - val_loss: 3.7621
Epoch 4/60
 - 0s - loss: 3.8356 - val_loss: 3.1432
Epoch 5/60
 - 0s - loss: 3.8871 - val_loss: 3.1514
Epoch 6/60
 - 0s - loss: 4.2217 - val_loss: 3.0456
Epoch 7/60
 - 0s - loss: 3.9083 - val_loss: 3.0345
Epoch 8/60
 - 0s - loss: 3.5532 - val_loss: 3.3100
Epoch 9/60
 - 0s - loss: 3.5468 - val_loss: 3.6179
Epoch 10/60
 - 0s - loss: 3.6818 - val_loss: 3.6613
Epoch 11/60
 - 0s - loss: 3.6613 - val_loss: 3.4341
Epoch 12/60
 - 0s - loss: 3.5261 - val_loss: 3.1674
Epoch 13/60
 - 0s - loss: 3.3976 - val_loss: 2.9394
Epoch 14/60
 - 0s - l

Epoch 32/60
 - 0s - loss: 0.6987 - val_loss: 0.7875
Epoch 33/60
 - 0s - loss: 0.6858 - val_loss: 0.7440
Epoch 34/60
 - 0s - loss: 0.6700 - val_loss: 0.6893
Epoch 35/60
 - 0s - loss: 0.6548 - val_loss: 0.6326
Epoch 36/60
 - 0s - loss: 0.6423 - val_loss: 0.5812
Epoch 37/60
 - 0s - loss: 0.6339 - val_loss: 0.5395
Epoch 38/60
 - 0s - loss: 0.6292 - val_loss: 0.5084
Epoch 39/60
 - 0s - loss: 0.6266 - val_loss: 0.4884
Epoch 40/60
 - 0s - loss: 0.6241 - val_loss: 0.4789
Epoch 41/60
 - 0s - loss: 0.6204 - val_loss: 0.4790
Epoch 42/60
 - 0s - loss: 0.6146 - val_loss: 0.4880
Epoch 43/60
 - 0s - loss: 0.6070 - val_loss: 0.5049
Epoch 44/60
 - 0s - loss: 0.5983 - val_loss: 0.5287
Epoch 45/60
 - 0s - loss: 0.5894 - val_loss: 0.5575
Epoch 46/60
 - 0s - loss: 0.5813 - val_loss: 0.5882
Epoch 47/60
 - 0s - loss: 0.5752 - val_loss: 0.6176
Epoch 48/60
 - 0s - loss: 0.5705 - val_loss: 0.6416
Epoch 49/60
 - 0s - loss: 0.5667 - val_loss: 0.6566
Epoch 50/60
 - 0s - loss: 0.5627 - val_loss: 0.6606
Epoch 51/60


Epoch 7/60
 - 0s - loss: 1.1854 - val_loss: 1.8395
Epoch 8/60
 - 0s - loss: 1.1321 - val_loss: 1.6963
Epoch 9/60
 - 0s - loss: 0.9515 - val_loss: 1.5825
Epoch 10/60
 - 0s - loss: 0.7902 - val_loss: 1.5511
Epoch 11/60
 - 0s - loss: 0.7074 - val_loss: 1.5887
Epoch 12/60
 - 0s - loss: 0.6991 - val_loss: 1.6571
Epoch 13/60
 - 0s - loss: 0.7332 - val_loss: 1.7248
Epoch 14/60
 - 0s - loss: 0.7771 - val_loss: 1.7710
Epoch 15/60
 - 0s - loss: 0.8101 - val_loss: 1.7883
Epoch 16/60
 - 0s - loss: 0.8224 - val_loss: 1.7773
Epoch 17/60
 - 0s - loss: 0.8130 - val_loss: 1.7426
Epoch 18/60
 - 0s - loss: 0.7864 - val_loss: 1.6922
Epoch 19/60
 - 0s - loss: 0.7503 - val_loss: 1.6364
Epoch 20/60
 - 0s - loss: 0.7137 - val_loss: 1.5845
Epoch 21/60
 - 0s - loss: 0.6852 - val_loss: 1.5448
Epoch 22/60
 - 0s - loss: 0.6705 - val_loss: 1.5208
Epoch 23/60
 - 0s - loss: 0.6713 - val_loss: 1.5110
Epoch 24/60
 - 0s - loss: 0.6833 - val_loss: 1.5096
Epoch 25/60
 - 0s - loss: 0.6981 - val_loss: 1.5094
Epoch 26/60
 - 

Epoch 43/60
 - 0s - loss: 1.4708 - val_loss: 1.0243
Epoch 44/60
 - 0s - loss: 1.4688 - val_loss: 1.0342
Epoch 45/60
 - 0s - loss: 1.4679 - val_loss: 1.0445
Epoch 46/60
 - 0s - loss: 1.4663 - val_loss: 1.0540
Epoch 47/60
 - 0s - loss: 1.4626 - val_loss: 1.0622
Epoch 48/60
 - 0s - loss: 1.4566 - val_loss: 1.0690
Epoch 49/60
 - 0s - loss: 1.4485 - val_loss: 1.0753
Epoch 50/60
 - 0s - loss: 1.4393 - val_loss: 1.0818
Epoch 51/60
 - 0s - loss: 1.4301 - val_loss: 1.0892
Epoch 52/60
 - 0s - loss: 1.4220 - val_loss: 1.0981
Epoch 53/60
 - 0s - loss: 1.4156 - val_loss: 1.1082
Epoch 54/60
 - 0s - loss: 1.4110 - val_loss: 1.1190
Epoch 55/60
 - 0s - loss: 1.4078 - val_loss: 1.1296
Epoch 56/60
 - 0s - loss: 1.4050 - val_loss: 1.1391
Epoch 57/60
 - 0s - loss: 1.4018 - val_loss: 1.1472
Epoch 58/60
 - 0s - loss: 1.3976 - val_loss: 1.1539
Epoch 59/60
 - 0s - loss: 1.3924 - val_loss: 1.1598
Epoch 60/60
 - 0s - loss: 1.3865 - val_loss: 1.1655
99
[99]
Train on 34 samples, validate on 9 samples
Epoch 1/60
 -

In [26]:
df= pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Splited Data s-1 to s-5/STEST/S_1__I_582865.csv",parse_dates=['date'])

#convert date field from string to datetime
#df['date'] = pd.to_datetime(df['date'])

In [27]:
df.drop(['id','store_nbr','item_nbr','onpromotion'], axis=1,inplace=True)

In [28]:
df=df[0:800]
k = int(len(df)*0.8)
train = df[:k]  
test= df[k:]
lag_size = (test['date'].max().date() - train['date'].max().date()).days

In [29]:
window =35
lag = lag_size
series = series_to_supervised(train.drop('date', axis=1), window=window, lag=lag)
labels_col = 'unit_sales(t+%d)' % lag
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

In [30]:
X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
#Y_valid=df[200:].unit_sales
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1],1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1],1))
subsequences = 2
timesteps = X_train_series.shape[1]//subsequences
X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, timesteps, 1))
X_valid_series_sub = X_valid_series.reshape((X_valid_series.shape[0], subsequences, timesteps, 1))
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation='relu'))
            #model_cnn_lstm.add(LSTM(10, stateful=True))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mse', optimizer='adam')
cnn_lstm_history = model_cnn_lstm.fit(X_train_series_sub, Y_train, validation_data=(X_valid_series_sub, Y_valid), epochs=60, verbose=2)
cnn_lstm_valid_pred = model_cnn_lstm.predict(X_valid_series_sub)

Train on 147 samples, validate on 99 samples
Epoch 1/60
 - 1s - loss: 62.1616 - val_loss: 27.9890
Epoch 2/60
 - 0s - loss: 25.2912 - val_loss: 37.4495
Epoch 3/60
 - 0s - loss: 23.5539 - val_loss: 26.5742
Epoch 4/60
 - 0s - loss: 23.4159 - val_loss: 29.2623
Epoch 5/60
 - 0s - loss: 23.1704 - val_loss: 32.5756
Epoch 6/60
 - 0s - loss: 21.9627 - val_loss: 26.8814
Epoch 7/60
 - 0s - loss: 22.0084 - val_loss: 27.0201
Epoch 8/60
 - 0s - loss: 21.3169 - val_loss: 28.8921
Epoch 9/60
 - 0s - loss: 21.2632 - val_loss: 28.1486
Epoch 10/60
 - 0s - loss: 20.9069 - val_loss: 27.7069
Epoch 11/60
 - 0s - loss: 20.7010 - val_loss: 28.0972
Epoch 12/60
 - 0s - loss: 20.5668 - val_loss: 28.7282
Epoch 13/60
 - 0s - loss: 20.5956 - val_loss: 27.7237
Epoch 14/60
 - 0s - loss: 20.4250 - val_loss: 28.4187
Epoch 15/60
 - 0s - loss: 19.7564 - val_loss: 27.4490
Epoch 16/60
 - 0s - loss: 20.2279 - val_loss: 28.6408
Epoch 17/60
 - 0s - loss: 19.7355 - val_loss: 27.4577
Epoch 18/60
 - 0s - loss: 19.3104 - val_loss: 

In [32]:
(cnn_lstm_valid_pred).shape

(99, 1)

In [31]:
X_train_series_sub.shape

(147, 2, 18, 1)

In [114]:
forecast1=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/ML_FORECAST_70%.csv")

In [115]:
forecast1['CNN+LSTM']=cnn_lstm_valid_pred[4:]

In [116]:
forecast1.to_csv('ML_FORECAST_60%.csv')

In [117]:
forecast1

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,date,actual_value,LSTM_1,ANN,CNN,RNN_2,CNN+LSTM
0,0,0,0,0,2013-08-20,15.910,12.966378,12.579720,16.562193,12.276929,11.162116
1,1,1,1,1,2013-08-21,10.261,12.850510,12.747951,14.999523,13.190181,14.462693
2,2,2,2,2,2013-08-22,15.035,13.017899,12.972117,16.905703,11.340872,11.675538
3,3,3,3,3,2013-08-23,12.190,12.846584,12.491053,13.831470,14.544037,11.480006
4,4,4,4,4,2013-08-24,11.514,12.917412,12.801822,7.564637,12.094693,11.405085
...,...,...,...,...,...,...,...,...,...,...,...
65,65,65,65,65,2013-10-24,13.996,12.674856,12.599476,6.641022,12.206569,12.572830
66,66,66,66,66,2013-10-25,15.341,12.811443,12.186004,12.548703,11.763103,12.719814
67,67,67,67,67,2013-10-26,9.206,12.779969,12.580894,15.209799,11.140363,13.942182
68,68,68,68,68,2013-10-27,12.771,13.015878,13.101615,14.770230,11.462310,15.413132


In [54]:
Y_valid

array([ 7.01 , 11.288, 22.559, 11.331, 21.092, 11.302,  2.743, 14.565,
       16.361, 15.969, 22.671, 15.304, 23.841, 16.482, 16.48 ,  4.131,
        7.669,  9.759, 12.816, 11.257, 12.52 , 11.769, 15.365, 20.088,
       11.947, 15.216, 10.245, 17.065, 13.924,  7.673, 12.053, 13.024,
       22.19 , 12.002, 10.416, 14.798,  3.049, 13.957, 15.727,  9.269,
       12.88 ,  5.165, 23.759, 13.798, 12.137,  9.604,  8.249,  6.79 ,
        5.712,  8.784,  9.255, 15.271, 15.767, 12.376, 16.596,  8.153,
       10.339, 11.39 , 20.431, 19.337, 10.914, 15.576, 15.496, 12.888,
       13.88 , 15.684, 24.069, 12.831, 26.017, 13.547, 13.129, 14.063,
       16.127,  6.261,  9.253, 16.576, 13.42 ,  3.565, 11.294, 10.804,
       13.78 ])

In [118]:
np.sqrt(mean_squared_error(Y_valid, cnn_lstm_valid_pred))

5.419344891177836

In [53]:
cnn_lstm_valid_pred

array([[ 9.749607 ],
       [16.521664 ],
       [15.808555 ],
       [13.0662775],
       [ 9.827027 ],
       [12.472345 ],
       [11.783607 ],
       [11.490197 ],
       [12.30871  ],
       [15.308165 ],
       [12.11848  ],
       [14.018382 ],
       [ 9.326438 ],
       [10.338875 ],
       [11.09764  ],
       [11.01768  ],
       [14.130488 ],
       [13.857646 ],
       [ 8.757605 ],
       [15.861598 ],
       [10.938604 ],
       [10.587022 ],
       [12.232128 ],
       [14.386208 ],
       [14.584804 ],
       [15.90261  ],
       [11.044084 ],
       [17.267014 ],
       [14.625699 ],
       [10.102806 ],
       [10.59043  ],
       [14.403614 ],
       [13.175626 ],
       [16.459545 ],
       [17.64472  ],
       [15.895705 ],
       [13.521489 ],
       [12.951949 ],
       [16.767649 ],
       [14.528293 ],
       [13.402775 ],
       [13.740217 ],
       [12.472003 ],
       [19.448648 ],
       [15.030196 ],
       [11.971045 ],
       [12.879219 ],
       [ 8.58

In [16]:
mean_squared_error(error['RMSE_ERROR_LSTM'], error['RMSE_ERROR_CNN'])

0.22490883212879847

In [28]:
len(error[error['RMSE_ERROR_LSTM']>error['RMSE_ERROR_CNN+LSTM']])

43

In [29]:
error.to_csv('CNN+LSTM_forecast_error.csv')